In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv("datasets/Heart Disease data.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [3]:
y=df["target"]
X= df.drop(columns=["target"])
y


0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: target, Length: 1025, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state =42,test_size=0.2)

In [5]:
from sklearn.metrics import accuracy_score
from sklearn import tree
import pickle




In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
model1 =LogisticRegression()
model2 = DecisionTreeClassifier()
model3 = KNeighborsClassifier()
models =[model1,model2,model3]


In [48]:
class Voting_Ensembler:
    def __init__(self,criterion,bagger,models):
        self.bagger =bagger
        self.criterion = criterion
        self.clf_list = models
        self.models=[]
        self.scores_list =[]

    def fit(self,X,y):
        for i in range(self.bagger):
            X_train_s,X_test_s,y_train_s,y_test_s=train_test_split(X,y,random_state=42,test_size=0.1)
            self.models.append(self.clf_list[i].fit(X_train_s,y_train_s))
            

    def save_model(self,filename):
        pickle.dump(self.models,open(filename,"wb"))
    def load_model(self,filename):
        loaded_model =pickle.load(open(filename,"rb"))
        self.models =loaded_model
        return loaded_model
    def predict(self,X_test):
        preds=[]
        for i in range(self.bagger):
            prediction = self.models[i].predict(X_test)
            preds.append(prediction)
        return preds
    def calculate_accuracy(self,preds,y_test):
        for i in range(self.bagger):
           self.scores_list.append(accuracy_score(preds[i],y_test))
        return self.scores_list
    
            
    

rf =Voting_Ensembler("entropy",3,models)


    

In [49]:
rf.fit(X_train,y_train)

C:\Users\PMLS\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
preds = rf.predict(X_test)

In [51]:
filename ="datasets/random_forest.pkl"
rf.save_model(filename)

# Load the models into the rf object
rf.load_model(filename)

# Predict using the loaded models
preds = rf.predict(X_test)

# Calculate accuracy based on the true labels
accuracies = rf.calculate_accuracy(preds, y_test)
print(accuracies)

[0.7951219512195122, 0.9902439024390244, 0.697560975609756]
